In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [14]:
class PositionalEmbedding(nn.Module):
    def __init__(self, ):
        super().__init__()
    
    def forward(self, x):
        return x

In [15]:
class PatchEmbedding(nn.Module):
    def __init__(self, ):
        super().__init__()
    
    def forward(self, x):
        return x

In [16]:
class Block(nn.Module):
    def __init__(self, ):
        super().__init__()
    
    def forward(self, x):
        return x

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, emb_dim,
                 patch_size,
                 in_channels,
                 img_size, 
                 n_classes,
                 n_blocks
                 ):
        super().__init__()
        self.n_classes = n_classes
        self.patch_emb = PatchEmbedding(
                 emb_dim, patch_size,
                 in_channels, img_size
                 )
        self.pos_emb = PositionalEmbedding()
        self.out_norm = nn.LayerNorm(emb_dim)
        self.out_proj = nn.Linear(emb_dim, self.n_classes)
        self.blocks = nn.Sequential(* [ Block() for _ in range(n_blocks) ] )
        self.cls = nn.Parameter( torch.zeros(1, ) ) 
    
    def forward(self, x, targets = None):
        print(x.shape)
        B = x.shape[0]
        x = self.patch_emd(x)
        
        if self.pooling_type == "cls":
            pass
        elif self.pooling_type == "avg":
            pass

        x = x + self.pos_emb(x)
        for block in self.blocks:
            x = block(x)
        
        x = self.out_norm(x)
        logits = self.out_proj(x)
        loss = None
        if targets is not None:
            pass
        
        return logits, loss


In [ ]:
model = VisionTransformer()
x = torch.randn( size=(2, 3, 224, 224) ) # (B, image channels, height, width)
model(x)

torch.Size([2, 3, 224, 224])
